In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
os.chdir('/Users/duanyihong/Desktop/financial markets/Assignment 7')

In [5]:
BLport=pd.DataFrame(pd.read_excel('BL_port.xlsx'))
Cov=pd.read_csv('Cov.csv')

## Derive the optimal portfolio weights

In [6]:
#Clean the data and transform it to a Covariance matrix
Cov_val=Cov[['Australia','Canada','France','Germany','Japan','UK','USA']]
Cov_val.index=Cov_val.columns

In [7]:
#Calculate the inverse of covariance matrix
Cov_inv=pd.DataFrame(np.linalg.pinv(Cov_val.values),Cov_val.columns,Cov_val.index)
#Set risk aversion as 2.5
result=2.5*np.dot(Cov_val,BLport['Equilibrium Portfolio Weight (%)'])

In [8]:
BLport['Equilibrium expected return(%)']=result
BLport

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium expected return(%)
0,Australia,16.0,1.6,3.941200
1,Canada,20.3,2.2,6.918900
2,France,24.8,5.2,8.360875
3,Germany,27.1,5.5,9.026275
4,Japan,21.0,11.6,4.299575
5,UK,20.0,12.4,6.769225
6,USA,18.7,61.5,7.565375


## Use CAPM to derive the optimal weights

In [9]:
#Calculate the betas for each country with respect to USA
CAPM=pd.DataFrame(columns=['Beta','Expected return%'],index=Cov_val.index)
#The USA equity risk premium is about 5% in 2019
#Risk-free rate is about 2.5%
for i in Cov_val.index:
    CAPM['Beta'].loc[i]=Cov_val['USA'].loc[i]/Cov_val['USA'].loc['USA']
    CAPM['Expected return%']=(CAPM['Beta']*0.05+0.025)*100

In [10]:
CAPM['Expected return%']

Australia        4.6
Canada       6.72857
France       6.92857
Germany      7.22857
Japan        4.21429
UK           5.98571
USA              7.5
Name: Expected return%, dtype: object

## Add a view and derive new estimates of the risk premia

In [11]:
#first form P matrix
#my view: Japan outperforms all the European countries by 5%
#P_prime=[0,0,-0.3,-0.4,1,-0.3,0]
#test the case in class that Germany will outperform the rest of Europe
P_prime=[0,0,-0.3,1,0,-0.7,0]
Q=0.05
#suppose my view has variance of 0.2^2
myvar=0.04
#set tao as 1: I'm not that confident about my result
tao=1
addi=np.dot(np.linalg.pinv(tao*Cov_val),BLport['Equilibrium expected return(%)'])+(1/myvar)*np.transpose(P_prime)*Q
Cov_new=np.linalg.pinv(np.linalg.pinv(tao*Cov_val)+(1/myvar)*np.dot(np.transpose(P_prime),P_prime))

In [12]:
BL_return=np.dot(Cov_new,addi)

In [13]:
BLport['R_BL']=BL_return

In [14]:
BLport['BL_weight/Original%']=np.dot(np.linalg.pinv(2.5*Cov_val),BL_return)

In [15]:
BLport['BL_weight/Updated%']=np.dot(np.linalg.pinv(2.5*Cov_new),BL_return)

In [16]:
BLport

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium expected return(%),R_BL,BL_weight/Original%,BL_weight/Updated%
0,Australia,16.0,1.6,3.941200,-0.321023,-11.666895,1.60
1,Canada,20.3,2.2,6.918900,0.495318,-11.066895,2.20
2,France,24.8,5.2,8.360875,-0.250561,-8.216895,5.05
3,Germany,27.1,5.5,9.026275,-0.389871,-7.266895,6.00
4,Japan,21.0,11.6,4.299575,-0.330576,-1.666895,11.60
5,UK,20.0,12.4,6.769225,0.026084,-1.216895,12.05
6,USA,18.7,61.5,7.565375,1.610306,48.233105,61.50


In [195]:
Cov_new

array([[ 0.01390179, -0.00173367, -0.00464184, -0.00359604,  0.00210194,
        -0.00208808, -0.00164296],
       [-0.00173367,  0.01461934, -0.00223727, -0.00303521, -0.00594082,
        -0.00316859,  0.00496493],
       [-0.00464184, -0.00223727,  0.01372034,  0.00556465, -0.00716251,
         0.00143597, -0.00202877],
       [-0.00359604, -0.00303521,  0.00556465,  0.01607458, -0.00800938,
         0.00117339, -0.003078  ],
       [ 0.00210194, -0.00594082, -0.00716251, -0.00800938,  0.03031664,
        -0.00306826, -0.00573979],
       [-0.00208808, -0.00316859,  0.00143597,  0.00117339, -0.00306826,
         0.01078107, -0.00142873],
       [-0.00164296,  0.00496493, -0.00202877, -0.003078  , -0.00573979,
        -0.00142873,  0.01216812]])